<a href="https://colab.research.google.com/github/miguelvelascop/PFG_Miguel_Velasco_Puig/blob/main/Entrenamiento_Modelos_PFG_Depresjon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import scipy as sc
import sklearn as sk
import pandas as pd
import matplotlib.pyplot as plt

Cargamos el dataset, depresión y no depresión por separado, que previamente he mergeado, ya que estaban separados por pacientes

In [ ]:
from google.colab import files
depresjon_data = files.upload()



In [ ]:
data=pd.read_csv("Dataset_Depresjon_procesado.csv",  sep=',',  comment='#')

Pasamos los csv a matrices, quitando los índices


In [ ]:
depresjon_origin = pd.read_csv("Dataset_Depresjon_procesado.csv",  sep=',',  comment='#').to_numpy()

Separar las etiquetas de los datos de entrenamiento

In [ ]:
depresjon_data = depresjon_origin[:, :-1]
depresjon_target = depresjon_origin[:, -1]

Oversampling SMOTE

In [ ]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=50)
depresjon_data_resampled, depresjon_target_resampled = smote.fit_resample(depresjon_data, depresjon_target)
print(depresjon_data_resampled.size)

Normalización

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler().fit(depresjon_data_resampled)
depresjon_data_resampled_rescaled = scaler.transform(depresjon_data_resampled)

Selección de caráccterísticas (no se utiliza posteriormente)

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

selector = SelectKBest(chi2, k=6)

print(depresjon_data_resampled_rescaled.shape)
test_seleccion_caracteristicas = selector.fit_transform(depresjon_data_resampled_rescaled, depresjon_target_resampled)
print(selector.get_support(indices=True))
print(selector.scores_)
print(test_seleccion_caracteristicas.shape)

Separación de los datos en 20/80 para tener un conjunto de entrenamiento y una de prueba

In [ ]:
from sklearn.model_selection import train_test_split
train_depresjon_x, test_depresjon_x, train_depresjon_y, test_depresjon_y = train_test_split(depresjon_data_resampled_rescaled, depresjon_target_resampled, test_size=0.2, random_state=42)

Primero importo los modelos que usaremos

In [ ]:
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.gaussian_process import GaussianProcessClassifier as GPC
from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.ensemble import AdaBoostClassifier as ABC
from sklearn.naive_bayes import GaussianNB as NB
from sklearn.neural_network import MLPClassifier as MLP

from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, precision_score, recall_score, f1_score

from sklearn.model_selection import GridSearchCV

In [ ]:
selector = SelectKBest(chi2, k=6)

print(train_depresjon_x.shape)
demonicSalmon_data = selector.fit_transform(demonicSalmon_data, demonicSalmon_target)
print(selector.get_support(indices=True))
print(selector.scores_)
print(demonicSalmon_data.shape)

Selección de parámetros KNN (el entrenamiento, la predicción y la obtención de métricas se realiza 10 veces)

In [ ]:
parametros = {'n_neighbors':[1, 10]}
knn = KNN()
knn = GridSearchCV(knn, parametros)
accuracyKNN = []
precisionKNN = []
recallKNN = []
f1KNN = []
for i in range(10):
  knn.fit(train_depresjon_x, train_depresjon_y)
  resultKNN = knn.predict(test_depresjon_x)
  accuracyKNN.append(accuracy_score(resultKNN, test_depresjon_y))
  precisionKNN.append(precision_score(resultKNN, test_depresjon_y))
  recallKNN.append(recall_score(resultKNN, test_depresjon_y))
  f1KNN.append(f1_score(resultKNN, test_depresjon_y))

Entrenamiento KNN

In [ ]:
resultKNN = knn.fit(train_depresjon_x, train_depresjon_y)


Validación KNN

In [ ]:
scoresKNN = cross_val_score(resultKNN, train_depresjon_x, train_depresjon_y, cv=10)
print(scoresKNN)

Matriz de confusión

In [ ]:
y_pred = knn.predict(test_depresjon_x)
cm = confusion_matrix(test_depresjon_y, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.show()

Média y varianza de KNN

In [ ]:
meanAcKNN = np.median(accuracyKNN)
stdAcKNN=np.std(accuracyKNN)
meanPrKNN = np.median(precisionKNN)
stdPrKNN=np.std(precisionKNN)
meanReKNN = np.median(recallKNN)
stdReKNN=np.std(recallKNN)
meanF1KNN = np.median(f1KNN)
stdF1KNN=np.std(f1KNN)

Selección de parámetros SVC

In [ ]:
parametros = {'kernel': ('linear', 'rbf'),'C': [1, 10, 100]}
svm = SVC()
accuracySVC = []
precisionSVC = []
recallSVC = []
f1SVC = []
svm = GridSearchCV(svm, parametros)
for i in range(10):
  svm.fit(train_depresjon_x, train_depresjon_y)
  resultSVC = svm.predict(test_depresjon_x)
  accuracySVC.append(accuracy_score(resultSVC, test_depresjon_y))
  precisionSVC.append(precision_score(resultSVC, test_depresjon_y))
  recallSVC.append(recall_score(resultSVC, test_depresjon_y))
  f1SVC.append(f1_score(resultSVC, test_depresjon_y))

Entrenamiento SVM

In [ ]:
resultSVM = svm.fit(train_depresjon_x, train_depresjon_y)

Validación SVM

In [ ]:
scoresSVM = cross_val_score(resultSVM, train_depresjon_x, train_depresjon_y, cv=10)
print(scoresSVM)

In [ ]:
y_pred = svm.predict(test_depresjon_x)
cm = confusion_matrix(test_depresjon_y, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.show()

Média y varianza de SVM

In [ ]:
meanAcSVC = np.median(accuracySVC)
stdAcSVC=np.std(accuracySVC)
meanPrSVC = np.median(precisionSVC)
stdPrSVC=np.std(precisionSVC)
meanReSVC = np.median(recallSVC)
stdReSVC=np.std(recallSVC)
meanF1SVC = np.median(f1SVC)
stdF1SVC=np.std(f1SVC)
print(meanAcSVC, stdAcSVC)

Selección de parámetros linearSVM

In [ ]:
parametros = {'C': [1, 10], 'penalty':('l1','l2'),'loss':('hinge','square_hinge'),'intercept_scaling':[1, 10]}
linearSVM = LinearSVC()
accuracyLinearSVM = []
precisionLinearSVM = []
recallLinearSVM = []
f1LinearSVM = []
linearSVM = GridSearchCV(linearSVM, parametros)
for i in range(10):
  linearSVM.fit(train_depresjon_x, train_depresjon_y)
  resultLinearSVM = linearSVM.predict(test_depresjon_x)
  accuracyLinearSVM.append(accuracy_score(resultLinearSVM, test_depresjon_y))
  precisionLinearSVM.append(precision_score(resultLinearSVM, test_depresjon_y))
  recallLinearSVM.append(recall_score(resultLinearSVM, test_depresjon_y))
  f1LinearSVM.append(f1_score(resultLinearSVM, test_depresjon_y))

Entrenamiento LinearSVM

In [ ]:
modelSVM = linearSVM.fit(train_depresjon_x, train_depresjon_y)

Validación LinearSVC

In [ ]:
scoresLinearSVM = cross_val_score(modelSVM, train_depresjon_x, train_depresjon_y, cv=10)
print(scoresLinearSVM)

In [ ]:
y_pred = linearSVM.predict(test_depresjon_x)
cm = confusion_matrix(test_depresjon_y, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.show()

Média y varianza de LinearSVC

In [ ]:
meanAcLinearSVM = np.median(accuracyLinearSVM)
stdAcLinearSVM=np.std(accuracyLinearSVM)
meanPrLinearSVM = np.median(precisionLinearSVM)
stdPrLinearSVM=np.std(precisionLinearSVM)
meanReLinearSVM = np.median(recallLinearSVM)
stdReLinearSVM=np.std(recallLinearSVM)
meanF1LinearSVM = np.median(f1LinearSVM)
stdF1LinearSVM=np.std(f1LinearSVM)
print(meanAcLinearSVM, stdAcLinearSVM)

Selección de parámetros GPC

In [ ]:
parametros = {'optimizer':('fmin_l_bfgs_b','callable','None')}
gpc = GPC()
accuracyGPC = []
precisionGPC = []
recallGPC = []
f1GPC = []
gpc = GridSearchCV(gpc, parametros)
for i in range(10):
  gpc.fit(train_depresjon_x, train_depresjon_y)
  resultGPC = gpc.predict(test_depresjon_x)
  accuracyGPC.append(accuracy_score(resultGPC, test_depresjon_y))
  precisionGPC.append(precision_score(resultGPC, test_depresjon_y))
  recallGPC.append(recall_score(resultGPC, test_depresjon_y))
  f1GPC.append(f1_score(resultGPC, test_depresjon_y))

Entrenamiento GPC

In [ ]:
modelGPC = gpc.fit(train_depresjon_x, train_depresjon_y)

Validación GPC

In [ ]:
scoresGPC = cross_val_score(modelGPC, train_depresjon_x, train_depresjon_y, cv=10)
print(scoresGPC)

Matriz de Confusión

In [ ]:
y_pred = gpc.predict(test_depresjon_x)
cm = confusion_matrix(test_depresjon_y, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.show()

Média y varianza de GPC

In [ ]:
meanAcGPC = np.median(accuracyGPC)
stdAcGPC=np.std(accuracyGPC)
meanPrGPC = np.median(precisionGPC)
stdPrGPC=np.std(precisionGPC)
meanReGPC = np.median(recallGPC)
stdReGPC=np.std(recallGPC)
meanF1GPC = np.median(f1GPC)
stdF1GPC=np.std(f1GPC)
print(meanAcGPC, stdAcGPC)

Selección de parámetros DTC

In [ ]:
parametros = {'min_samples_split':[1,100],'splitter':('best','random'),'max_features':('auto','sqrt')}
accuracyDTC = []
precisionDTC = []
recallDTC = []
f1DTC = []
dtc = DTC()
dtc = GridSearchCV(dtc, parametros)
for i in range(10):
  dtc.fit(train_depresjon_x, train_depresjon_y)
  resultDTC = dtc.predict(test_depresjon_x)
  accuracyDTC.append(accuracy_score(resultDTC, test_depresjon_y))
  precisionDTC.append(precision_score(resultDTC, test_depresjon_y))
  recallDTC.append(recall_score(resultDTC, test_depresjon_y))
  f1DTC.append(f1_score(resultDTC, test_depresjon_y))

Entrenamiento DTC

In [ ]:
resultDTC = dtc.fit(train_depresjon_x, train_depresjon_y)

Validación DTC

In [ ]:
scoresDTC = cross_val_score(resultDTC, train_depresjon_x, train_depresjon_y, cv=10)
print(scoresDTC)

Matriz de Confusión

In [ ]:
y_pred = dtc.predict(test_depresjon_x)
cm = confusion_matrix(test_depresjon_y, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.show()

Média y varianza de DTC

In [ ]:
meanAcDTC = np.median(accuracyDTC)
stdAcDTC=np.std(accuracyDTC)
meanPrDTC = np.median(precisionDTC)
stdPrDTC=np.std(precisionDTC)
meanReDTC = np.median(recallDTC)
stdReDTC=np.std(recallDTC)
meanF1DTC = np.median(f1DTC)
stdF1DTC=np.std(f1DTC)
print(meanAcDTC, stdAcDTC)

Seleccionar caracteríticas RFC

In [ ]:
parametros = {'criterion':('gini','entropy','log_loss'),'bootstrap':[True, False]}
accuracyRFC = []
precisionRFC = []
recallRFC = []
f1RFC = []
rfc = RFC()
rfc = GridSearchCV(rfc, parametros)
for i in range(10):
  rfc.fit(train_depresjon_x, train_depresjon_y)
  resultRFC = rfc.predict(test_depresjon_x)
  accuracyRFC.append(accuracy_score(resultRFC, test_depresjon_y))
  precisionRFC.append(precision_score(resultRFC, test_depresjon_y))
  recallRFC.append(recall_score(resultRFC, test_depresjon_y))
  f1RFC.append(f1_score(resultRFC, test_depresjon_y))

Entrenamiento RFC

In [ ]:
resultRFC = rfc.fit(train_depresjon_x, train_depresjon_y)

Validación RFC

In [ ]:
scoresRFC = cross_val_score(resultRFC, test_depresjon_x, test_depresjon_y, cv=10)
print(scoresRFC)

Matriz de Confusión

In [ ]:
y_pred = rfc.predict(test_depresjon_x)
cm = confusion_matrix(test_depresjon_y, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.show()

Média y varianza de RFC

In [ ]:
meanAcRFC = np.median(accuracyRFC)
stdAcRFC=np.std(accuracyRFC)
meanPrRFC = np.median(precisionRFC)
stdPrRFC=np.std(precisionRFC)
meanReRFC = np.median(recallRFC)
stdReRFC=np.std(recallRFC)
meanF1RFC = np.median(f1RFC)
stdF1RFC=np.std(f1RFC)
print(meanAcRFC, stdAcRFC)

Selección de características AdaBoost

In [ ]:
parametros = {'n_estimators':[20, 100],'algorithm':('SAMME', 'SAMME.R'),'learning_rate':[1, 10]}
accuracyABC = []
precisionABC = []
recallABC = []
f1ABC = []
abc = ABC()
abc = GridSearchCV(abc, parametros)
for i in range(10):
  abc.fit(train_depresjon_x, train_depresjon_y)
  resultABC = abc.predict(test_depresjon_x)
  accuracyABC.append(accuracy_score(resultABC, test_depresjon_y))
  precisionABC.append(precision_score(resultABC, test_depresjon_y))
  recallABC.append(recall_score(resultABC, test_depresjon_y))
  f1ABC.append(f1_score(resultABC, test_depresjon_y))

Entrenamiento AdaBoost

In [ ]:
resultABC = abc.fit(train_depresjon_x, train_depresjon_y)

Validación ABC

In [ ]:
scoresABC = cross_val_score(resultABC, test_depresjon_x, test_depresjon_y, cv=10)
print(scoresABC)

Matriz de Confusión

In [ ]:
y_pred = abc.predict(test_depresjon_x)
cm = confusion_matrix(test_depresjon_y, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.show()

Média y varianza de ABC

In [ ]:
meanAcABC = np.median(accuracyABC)
stdAcABC=np.std(accuracyABC)
meanPrABC = np.median(precisionABC)
stdPrABC=np.std(precisionABC)
meanReABC = np.median(recallABC)
stdReABC=np.std(recallABC)
meanF1ABC = np.median(f1ABC)
stdF1ABC=np.std(f1ABC)
print(meanAcABC, stdAcABC)

In [ ]:
nb = NB()
accuracyNB = []
precisionNB = []
recallNB = []
f1NB = []
for i in range(10):
  nb.fit(train_depresjon_x, train_depresjon_y)
  resultNB = nb.predict(test_depresjon_x)
  accuracyNB.append(accuracy_score(resultNB, test_depresjon_y))
  precisionNB.append(precision_score(resultNB, test_depresjon_y))
  recallNB.append(recall_score(resultNB, test_depresjon_y))
  f1NB.append(f1_score(resultNB, test_depresjon_y))

Entrenamiento GaussianNN

In [ ]:
resultNB = nb.fit(train_depresjon_x, train_depresjon_y)

Validación NB

In [ ]:
scoresNB = cross_val_score(resultNB, test_depresjon_x, test_depresjon_y, cv=10)
print(scoresNB)

Matriz de confusión

In [ ]:
y_pred = nb.predict(test_depresjon_x)
cm = confusion_matrix(test_depresjon_y, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.show()

Média y varianza de NB

In [ ]:
meanAcNB = np.median(accuracyNB)
stdAcNB=np.std(accuracyNB)
meanPrNB = np.median(precisionNB)
stdPrNB=np.std(precisionNB)
meanReNB = np.median(recallNB)
stdReNB=np.std(recallNB)
meanF1NB = np.median(f1NB)
stdF1NB=np.std(f1NB)
print(meanAcNB, stdAcNB)

Entrenamiento Red Neuronal (MLP)

In [ ]:
mlp = MLP(hidden_layer_sizes=(150,100,50),max_iter=300,activation='relu',solver='adam')
accuracyMLP = []
precisionMLP = []
recallMLP = []
f1MLP = []
for i in range(10):
  mlp.fit(train_depresjon_x, train_depresjon_y)
  resultMLP = mlp.predict(test_depresjon_x)
  accuracyMLP.append(accuracy_score(resultMLP, test_depresjon_y))
  precisionMLP.append(precision_score(resultMLP, test_depresjon_y))
  recallMLP.append(recall_score(resultMLP, test_depresjon_y))
  f1MLP.append(f1_score(resultMLP, test_depresjon_y))

Entrenamiento MLP

In [ ]:
resultMLP = mlp.fit(train_depresjon_x, train_depresjon_y)

Validación MLP

In [ ]:
scoresMLP = cross_val_score(resultMLP, test_depresjon_x, test_depresjon_y, cv=10)
print(scoresMLP)

Matriz de confusión MLP

In [ ]:
y_pred = mlp.predict(test_depresjon_x)
cm = confusion_matrix(test_depresjon_y, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.show()

Média y varianza de MLP

In [ ]:
meanMLP=np.median(accuracyMLP)
stdMLP=np.std(accuracyMLP)
meanAcMLP = np.median(accuracyMLP)
stdAcMLP=np.std(accuracyMLP)
meanPrMLP = np.median(precisionMLP)
stdPrMLP=np.std(precisionMLP)
meanReMLP = np.median(recallMLP)
stdReMLP=np.std(recallMLP)
meanF1MLP = np.median(f1MLP)
stdF1MLP=np.std(f1MLP)
print(meanAcMLP, stdAcMLP)

Comparación final

In [ ]:
print("Resultados KNN/ Accuracy: ",meanAcKNN, stdAcKNN, " Precision", meanPrKNN, stdPrKNN, " Recall:", meanReKNN, stdReKNN, " F1:",meanF1KNN, stdF1KNN)
print("Resultados SVM/ Accuracy: ",meanAcSVC, stdAcSVC," Precision",meanPrSVC, stdPrSVC," Recall:",meanReSVC, stdReSVC," F1:",meanF1SVC, stdF1SVC)
print("Resultados LSVM/ Accuracy: ",meanAcLinearSVM, stdAcLinearSVM," Precision",meanPrLinearSVM, stdPrLinearSVM," Recall:",meanReLinearSVM, stdReLinearSVM," F1:",meanF1LinearSVM, stdF1LinearSVM)
print("Resultados GPC/ Accuracy: ",meanAcGPC, stdAcGPC," Precision",meanPrGPC, stdPrGPC," Recall:",meanReGPC, stdReGPC," F1:",meanF1GPC, stdF1GPC)
print("Resultados DTC/ Accuracy: ",meanAcDTC, stdAcDTC," Precision",meanPrDTC, stdPrDTC," Recall:",meanReDTC, stdReDTC," F1:",meanF1DTC, stdF1DTC)
print("Resultados RFC/ Accuracy: ",meanAcRFC, stdAcRFC," Precision",meanPrRFC, stdPrRFC," Recall:",meanReRFC, stdReRFC," F1:",meanF1RFC, stdF1RFC)
print("Resultados AdaBoost/ Accuracy: :",meanAcABC, stdAcABC," Precision",meanPrABC, stdPrABC," Recall:",meanReABC, stdReABC," F1:",meanF1ABC, stdF1ABC)
print("Resultados GaussianNB/ Accuracy: ",meanAcNB, stdAcNB," Precision",meanPrNB, stdPrNB," Recall:",meanReNB, stdReNB," F1:",meanF1NB, stdF1NB)
print("Resultados MLP/ Accuracy: ",meanAcMLP, stdAcMLP," Precision",meanPrMLP, stdPrMLP," Recall:",meanReMLP, stdReMLP," F1:",meanF1MLP, stdF1MLP)